In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3



# Load data

In [ ]:
df_single = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_single_ko.pkl')
df_double = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_double.pkl')

# Model training on t=0

In [ ]:
cols = ['cellMass', 'growth', 'dryMass', 'waterMass', 'dnaMass', 'cytosol_mass',
       'tRnaMass', 'extracellular_mass', 'rRnaMass', 'proteinMass',
       'projection_mass', 'rnaMass', 'outer_membrane_mass', 'flagellum',
       'pilus_mass', 'cellVolume', 'inner_membrane_mass', 'mRnaMass',
       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
       'periplasm_mass']

#this is for the cells that survive only for 1 time-step
df_features2 = df_single[df_single['cellMass'].str.len().isna()]
    

#this is for the cells that survive for more than 1 time-step
df_features1 = df_single[~df_single['cellMass'].str.len().isna()]

for c in cols:
    df_features1[c] = df_features1[c].apply(lambda x:x[0])


        
df_features = pd.concat([df_features1, df_features2])
df_features['growth'] = df_features['growth'].fillna(0)
df_features = df_features.drop(columns=['processMassDifferences', 'time'])

In [ ]:
df_features.columns

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

y = df_features.label_death.replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
X = df_features.drop(['label', 'cell_path', 'generation', 'seed', 'instantaniousGrowthRate', 'cell', 'gene_name', 'label_lifetime', 'label_death', 'growth'], axis=1)

gs = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=6)

gen = list(next(gs.split(X, y, groups=X.gene_id)))
train_ix = gen[0]
test_ix = gen[1]

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

In [ ]:
y_train.value_counts()

In [ ]:
map_label = {'non_essential': 0, 'essential':1}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression


gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []

m_depth = [11, 13, 15, 17, 19, 21]
n_est = [22, 24, 26, 28]
m_leaf = [15, 16, 17, 18, 19]
comb = [(i, j, z) for i in m_depth for j in n_est for z in m_leaf]
res = []

for k in comb:
    clf = RandomForestClassifier(max_depth=k[0], n_estimators=k[1], random_state=0, max_leaf_nodes=k[2])
    for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=['gene_id']).astype(float), groups=X_train['gene_id'])):
        X_tr = X_train.drop(columns=[ 'gene_id']).iloc[ix_train]
        y_tr = y_train.iloc[ix_train]
        X_te = X_train.drop(columns=[ 'gene_id']).iloc[ix_test]
        y_te = y_train.iloc[ix_test]
        clf.fit(X_tr, y_tr)
        cnf1 = confusion_matrix(y_tr, clf.predict(X_tr))
        disp = ConfusionMatrixDisplay(confusion_matrix=cnf1,
                                   display_labels=clf.classes_)
        
        f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label='essential'))
        cnf2 = confusion_matrix(y_te, clf.predict(X_te))
        disp = ConfusionMatrixDisplay(confusion_matrix=cnf2,
                                   display_labels=clf.classes_)
        
        f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label='essential'))
    
    print('Avg eval ', sum(f1_eval)/len(f1_eval))
    
    clf.fit(X_train.drop(columns=['gene_id']), y_train)
    
    print('F1 score training ', f1_score(y_train, clf.predict(X_train.drop(columns=[ 'gene_id'])),  pos_label='essential'))
        
    print('F1 score testing', f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])),  pos_label='essential'))
    print(k)
    print('===========================')
    res.append([f1_score(y_train, clf.predict(X_train.drop(columns=[ 'gene_id'])),  pos_label='essential'), 
                f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])),  pos_label='essential'), k])


In [ ]:
sorted_list = sorted(res, key=lambda x: x[1], reverse=True)
sorted_list

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier


gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []
m_depth = [6, 8, 10, 12]
min_child_w = [18, 20, 22, 24]
n_estimators = [20, 30, 40, 50]
comb = [(i, j, z) for i in m_depth for j in min_child_w for z in n_estimators]
res = []

for k in comb:
    clf = xgb.XGBClassifier(max_depth=k[0], min_child_weight=k[1], n_estimators=k[2], learning_rate=0.1, verbosity=0)
    for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=['gene_id']).astype(float), groups=X_train['gene_id'])):
        X_train = X.iloc[train_ix]
        y_train = y.iloc[train_ix]

        X_test = X.iloc[test_ix]
        y_test = y.iloc[test_ix]
        
        X_tr = X_train.drop(columns=['gene_id']).iloc[ix_train].astype(float)
        y_tr = y_train.iloc[ix_train]
        X_te = X_train.drop(columns=['gene_id']).iloc[ix_test].astype(float)
        y_te = y_train.iloc[ix_test]
        y_tr = [map_label[x] for x in y_tr]
        y_te = [map_label[x] for x in y_te]
        clf.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=0)
    
        f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label=1))
        
        f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label=1))
    
    print(sum(f1_eval)/len(f1_eval))
    
    y_train = [map_label[x] for x in y_train]
    y_test = [map_label[x] for x in y_test]
    
    clf.fit(X_train.drop(columns=['gene_id']).astype(float), y_train, eval_set=[(X_test.drop(columns=['gene_id']).astype(float), y_test)], verbose=0)
    
    print(f1_score(y_train, clf.predict(X_train.drop(columns=['gene_id']).astype(float)),  pos_label=1))
        
    
    print(f1_score(y_test, clf.predict(X_test.drop(columns=['gene_id']).astype(float)),  pos_label=1))
    print(k)
    print('=======================')
    res.append([f1_score(y_train, clf.predict(X_train.drop(columns=[ 'gene_id']).astype(float)),  pos_label=1), 
                f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id']).astype(float)),  pos_label=1), k])




In [ ]:
sorted_list = sorted(res, key=lambda x: x[1], reverse=True)
sorted_list

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]
X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
    for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=[ 'gene_id']).astype(float), groups=X_train['gene_id'])):
        clf = KNeighborsClassifier(n_neighbors=k)
        X_tr = X_train.drop(columns=[ 'gene_id']).iloc[ix_train]
        y_tr = y_train.iloc[ix_train]
        X_te = X_train.drop(columns=[ 'gene_id']).iloc[ix_test]
        y_te = y_train.iloc[ix_test]
        clf.fit(X_tr, y_tr)
        
        f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label='essential'))
        f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label='essential'))

    clf.fit(X_train.drop(columns=['gene_id']), y_train)
    print(f1_score(y_train, clf.predict(X_train.drop(columns=['gene_id'])),  pos_label='essential'))
        
    cnf_test = confusion_matrix(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])))
    disp = ConfusionMatrixDisplay(confusion_matrix=cnf_test,
                                   display_labels=clf.classes_)
    print(f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])),  pos_label='essential'))
    
    FP = cnf_test.sum(axis=0) - np.diag(cnf_test)  
    FN = cnf_test.sum(axis=1) - np.diag(cnf_test)
    TP = np.diag(cnf_test)
    TN = cnf_test.sum() - (FP + FN + TP)
    print(k)
    print('False negative rate for testing is ' + str( FN/(TP+FN)))


# Test final single KOs trained RF on all features

In [ ]:
cols = ['cellMass', 'growth', 'dryMass', 'waterMass', 'dnaMass', 'cytosol_mass',
       'tRnaMass', 'extracellular_mass', 'rRnaMass', 'proteinMass',
       'projection_mass', 'rnaMass', 'outer_membrane_mass', 'flagellum',
       'pilus_mass', 'cellVolume', 'inner_membrane_mass', 'mRnaMass',
       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
       'periplasm_mass']

#this is for the cells that survive only for 1 time-step
df_features2 = df_single[df_single['cellMass'].str.len().isna()]
    

#this is for the cells that survive for more than 1 time-step
df_features1 = df_single[~df_single['cellMass'].str.len().isna()]

for c in cols:
    df_features1[c] = df_features1[c].apply(lambda x:x[-1])


        
df_features = pd.concat([df_features1, df_features2])
df_features['growth'] = df_features['growth'].fillna(0)
df_features = df_features.drop(columns=['processMassDifferences', 'time'])

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

y = df_features.label_death.replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
X = df_features.drop([ 'label', 'cell_path', 'generation', 'seed', 'instantaniousGrowthRate',
                       'cell', 'gene_name', 'label_lifetime', 'label_death', 'growth'], axis=1)

gs = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=6)

gen = list(next(gs.split(X, y, groups=X.gene_id)))
train_ix = gen[0]
test_ix = gen[1]

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

In [ ]:
y_train.value_counts()

In [ ]:
gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []

m_depth = [3, 4, 6, 8, 11, 13]
n_est = [4, 6, 8, 9, 14]
m_leaf = [10, 12, 15]
comb = [(i, j, z) for i in m_depth for j in n_est for z in m_leaf]
res = []

for k in comb:
    clf = RandomForestClassifier(max_depth=k[0], n_estimators=k[1], random_state=0, max_leaf_nodes=k[2])
    for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=['gene_id']).astype(float), groups=X_train['gene_id'])):
        X_tr = X_train.drop(columns=[ 'gene_id']).iloc[ix_train]
        y_tr = y_train.iloc[ix_train]
        X_te = X_train.drop(columns=[ 'gene_id']).iloc[ix_test]
        y_te = y_train.iloc[ix_test]
        clf.fit(X_tr, y_tr)
        cnf1 = confusion_matrix(y_tr, clf.predict(X_tr))
        disp = ConfusionMatrixDisplay(confusion_matrix=cnf1,
                                   display_labels=clf.classes_)
        
        f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label='essential'))
        cnf2 = confusion_matrix(y_te, clf.predict(X_te))
        disp = ConfusionMatrixDisplay(confusion_matrix=cnf2,
                                   display_labels=clf.classes_)
        
        f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label='essential'))
    
    print('Avg eval ', sum(f1_eval)/len(f1_eval))
    
    clf.fit(X_train.drop(columns=['gene_id']), y_train)
    
    print('F1 score training ', f1_score(y_train, clf.predict(X_train.drop(columns=[ 'gene_id'])),  pos_label='essential'))
        
    print('F1 score testing', f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])),  pos_label='essential'))
    print(k)
    print('===========================')
    res.append([f1_score(y_train, clf.predict(X_train.drop(columns=[ 'gene_id'])),  pos_label='essential'), 
                f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])),  pos_label='essential'), k])


In [ ]:
sorted_list = sorted(res, key=lambda x: x[1], reverse=True)
sorted_list

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier


gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []
m_depth = [6, 8, 10, 12]
min_child_w = [18, 20, 22, 24]
n_estimators = [20, 30, 40, 50]
comb = [(i, j, z) for i in m_depth for j in min_child_w for z in n_estimators]
res = []

for k in comb:
    clf = xgb.XGBClassifier(max_depth=k[0], min_child_weight=k[1], n_estimators=k[2], learning_rate=0.1, verbosity=0)
    for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=['gene_id']).astype(float), groups=X_train['gene_id'])):
        X_train = X.iloc[train_ix]
        y_train = y.iloc[train_ix]

        X_test = X.iloc[test_ix]
        y_test = y.iloc[test_ix]
        
        X_tr = X_train.drop(columns=['gene_id']).iloc[ix_train].astype(float)
        y_tr = y_train.iloc[ix_train]
        X_te = X_train.drop(columns=['gene_id']).iloc[ix_test].astype(float)
        y_te = y_train.iloc[ix_test]
        y_tr = [map_label[x] for x in y_tr]
        y_te = [map_label[x] for x in y_te]
        clf.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=0)
    
        f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label=1))
        
        f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label=1))
    
    print(sum(f1_eval)/len(f1_eval))
    
    y_train = [map_label[x] for x in y_train]
    y_test = [map_label[x] for x in y_test]
    
    clf.fit(X_train.drop(columns=['gene_id']).astype(float), y_train, eval_set=[(X_test.drop(columns=['gene_id']).astype(float), y_test)], verbose=0)
    
    print(f1_score(y_train, clf.predict(X_train.drop(columns=['gene_id']).astype(float)),  pos_label=1))
        
    
    print(f1_score(y_test, clf.predict(X_test.drop(columns=['gene_id']).astype(float)),  pos_label=1))
    print(k)
    print('=======================')
    res.append([f1_score(y_train, clf.predict(X_train.drop(columns=[ 'gene_id']).astype(float)),  pos_label=1), 
                f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id']).astype(float)),  pos_label=1), k])




In [ ]:
sorted_list = sorted(res, key=lambda x: x[1], reverse=True)
sorted_list

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]
X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
    for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=[ 'gene_id']).astype(float), groups=X_train['gene_id'])):
        clf = KNeighborsClassifier(n_neighbors=k)
        X_tr = X_train.drop(columns=[ 'gene_id']).iloc[ix_train]
        y_tr = y_train.iloc[ix_train]
        X_te = X_train.drop(columns=[ 'gene_id']).iloc[ix_test]
        y_te = y_train.iloc[ix_test]
        clf.fit(X_tr, y_tr)
        
        f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label='essential'))
        f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label='essential'))

    clf.fit(X_train.drop(columns=['gene_id']), y_train)
    print(f1_score(y_train, clf.predict(X_train.drop(columns=['gene_id'])),  pos_label='essential'))
        
    cnf_test = confusion_matrix(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])))
    disp = ConfusionMatrixDisplay(confusion_matrix=cnf_test,
                                   display_labels=clf.classes_)
    print(f1_score(y_test, clf.predict(X_test.drop(columns=[ 'gene_id'])),  pos_label='essential'))
    
    FP = cnf_test.sum(axis=0) - np.diag(cnf_test)  
    FN = cnf_test.sum(axis=1) - np.diag(cnf_test)
    TP = np.diag(cnf_test)
    TN = cnf_test.sum() - (FP + FN + TP)
    print(k)
    print('False negative rate for testing is ' + str( FN/(TP+FN)))


# Perform feature engineering

In [ ]:
cols = ['cellMass', 'growth', 'dryMass', 'waterMass', 'dnaMass', 'cytosol_mass',
       'tRnaMass', 'extracellular_mass', 'rRnaMass', 'proteinMass',
       'projection_mass', 'rnaMass', 'outer_membrane_mass', 'flagellum',
       'pilus_mass', 'cellVolume', 'inner_membrane_mass', 'mRnaMass',
       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
       'periplasm_mass']

#this is for the cells that survive only for 1 time-step
df_features2 = df_single[df_single['cellMass'].str.len().isna()]
for c in cols:
    df_features2[c] = df_features2[c].astype(float)
    

#this is for the cells that survive for more than 1 time-step
df_features1 = df_single[~df_single['cellMass'].str.len().isna()]

for c in cols:
    df_features1[c] = df_features1[c].apply(lambda x:x[-1])


        
df_features = pd.concat([df_features1,df_features2])
df_features['growth'] = df_features['growth'].fillna(0)
df_features = df_features.drop(columns=['processMassDifferences', 'time'])

In [ ]:
import seaborn as sn

corr_matrix = df_features.drop(columns=['label', 'cell_path', 'cell',
       'gene_id', 'seed', 'label_death', 
       'label_lifetime', 'gene_name']).corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df_features = df_features.drop(to_drop, axis=1)


fig, ax = plt.subplots(figsize=(20,20))  
svm = sn.heatmap(corr_matrix, annot=True,ax =ax,annot_kws={"size":16})

plt.show()
figure = svm.get_figure() 
figure.savefig('corr_matrix.png',bbox_inches="tight")

In [ ]:
len(to_drop)

# RF classifier

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
plt.rcParams["figure.figsize"] = (1.8, 2)

X = df_features.drop([ 'label', 'cell', 'seed', 'instantaniousGrowthRate',
                       'label_lifetime', 'generation'], axis=1)
X.label_death = X.label_death.replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
#based on an intial RF model, the following features seemed the most important to correctly classify
X = X[['cellMass', 'dnaMass', 'membrane_mass', 'extracellular_mass', 'gene_id', 'cell_path', 'label_death', 'gene_name']]
y=df_features.label_death.replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
gs = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=6)

gen = list(next(gs.split(X, y, groups=X.gene_id)))
train_ix = gen[0]
test_ix = gen[1]

X_train = X.iloc[train_ix].drop(columns=['label_death'])
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix].drop(columns=['label_death'])
y_test = y.iloc[test_ix]

In [ ]:
set(y_train)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
with open("trainig_genes.csv", "w") as output:
    output.write(str(list(set(X_train['gene_name']))))
with open("testing_genes.csv", "w") as output:
    output.write(str(list(set(X_test['gene_name']))))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
import matplotlib
colors = matplotlib.colors.ListedColormap(matplotlib.cm.get_cmap("tab20").colors[:4])

gkf_cv = GroupKFold(n_splits=5)
f1_tr = []
f1_eval = []
for split, (ix_train, ix_test) in enumerate(gkf_cv.split(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']).astype(float), groups=X_train['gene_id'])):
    

    clf = RandomForestClassifier(max_depth=4, n_estimators=3, random_state=0)
    X_tr = X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']).iloc[ix_train]
    y_tr = y_train.iloc[ix_train]
    X_te = X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']).iloc[ix_test]
    y_te = y_train.iloc[ix_test]
    clf.fit(X_tr, y_tr)
    cnf1 = confusion_matrix(y_tr, clf.predict(X_tr))
    disp = ConfusionMatrixDisplay(confusion_matrix=cnf1,
                               display_labels=clf.classes_)
    f1_tr.append(f1_score(y_tr, clf.predict(X_tr), pos_label='essential'))
    cnf2 = confusion_matrix(y_te, clf.predict(X_te))
    disp = ConfusionMatrixDisplay(confusion_matrix=cnf2,
                               display_labels=clf.classes_)
    f1_eval.append(f1_score(y_te, clf.predict(X_te), pos_label='essential'))

print('AVG cross-validation F1-score ', sum(f1_eval)/len(f1_eval))
clf.fit(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']), y_train)
print('Confussion matrix for the training set')
cnf_tr = confusion_matrix(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name'])))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_tr,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
print(cnf_tr)
disp.plot(cmap=colors, xticks_rotation='vertical')
plt.show()
disp.figure_.savefig('confusion_matrix_training_init.png', bbox_inches="tight", dpi=600)

print('Training data F1-score ', f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name'])),  pos_label='essential'))
    
print('Confusion matrix for the test set')
cnf_test = confusion_matrix(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id', 'gene_name'])))
print(cnf_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_test,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
disp.plot(cmap=colors, xticks_rotation='vertical')
plt.show()
disp.figure_.savefig('confusion_matrix_test_init.png', bbox_inches="tight", dpi=600)
print('F1 score for the test set is ' + str(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id', 'gene_name'])),  pos_label='essential')))

In [ ]:
plt.rcParams["figure.figsize"] = (1.8, 2)
features = X_train.columns
f_i = list(zip(features, clf.feature_importances_))
f_i.sort(key = lambda x : x[1], reverse=False)

plt.barh(*zip(*f_i), )
plt.xticks(rotation = 'horizontal',fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Feature importance', fontsize=12)
plt.savefig('feat_importance_rf_init.png', bbox_inches="tight")
plt.savefig('feat_importance_rf_init.eps', bbox_inches="tight", format='eps')
plt.show()

In [ ]:
import shap
# compute SHAP values
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']))

shap.summary_plot(shap_values[0], X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']).values,   feature_names = list(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']).columns), show=False)

plt.savefig('shap_init.png', dpi=600,bbox_inches="tight")
plt.savefig('shap_init.eps', bbox_inches="tight", format='eps')

In [ ]:
#Check which points are miss-classified
predictions = clf.predict(X_train.drop(columns=['cell_path', 'gene_id', 'gene_name']).values)
inputs = X_train['cell_path']
labels=y_train
wrong_class_gene = []
for input, prediction, label in zip(inputs, predictions, labels):
    if prediction != label:
        try:
            wrong_class_gene.append(input.split('/')[5].split('_')[3])
            print(input.split('/')[5].split('_')[3], 'has been classified as ', prediction, 'and should be ', label)
        except:
            wrong_class_gene.append(input.split('/')[9].split('_')[3])
            print(input.split('/')[9].split('_')[3], 'has been classified as ', prediction, 'and should be ', label)

                   
    

In [ ]:
#Check which points are miss-classified
predictions = clf.predict(X_test.drop(columns=['cell_path', 'gene_id', 'gene_name']).values)
inputs = X_test['cell_path']
labels=y_test
wrong_class_gene = []
for input, prediction, label in zip(inputs, predictions, labels):
    if prediction != label:
        try:
            wrong_class_gene.append(input.split('/')[5].split('_')[3])
            print(input.split('/')[5].split('_')[3], 'has been classified as ', prediction, 'and should be ', label)
        except:
            wrong_class_gene.append(input.split('/')[9].split('_')[3])
            print(input.split('/')[9].split('_')[3], 'has been classified as ', prediction, 'and should be ', label)

                   
    

In [ ]:
# save the model to disk
filename = 'rf_model_surrogate.sav'
pickle.dump(clf, open(filename, 'wb'))

## Test on double KOs

In [ ]:
cols = ['cellMass', 'growth', 'dryMass', 'waterMass', 'dnaMass', 'cytosol_mass',
       'tRnaMass', 'extracellular_mass', 'rRnaMass', 'proteinMass',
       'projection_mass', 'rnaMass', 'outer_membrane_mass', 'flagellum',
       'pilus_mass', 'cellVolume', 'inner_membrane_mass', 'mRnaMass',
       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
       'periplasm_mass']

#this is for the cells that survive only for 1 time-step
df_double1 = df_double[df_double['cellMass'].str.len().isna()]
for c in cols:
    df_double1[c] = df_double1[c].astype(float)
    

#this is for the cells that survive for more than 1 time-step
df_double2 = df_double[~df_double['cellMass'].str.len().isna()]

for c in cols:
    df_double2[c] = df_double2[c].apply(lambda x:x[-1])


        
df_double_feat = pd.concat([df_double1, df_double2])
df_double_feat['growth'] = df_double_feat['growth'].fillna(0)
df_double_feat = df_double_feat.drop(columns=['processMassDifferences', 'time'])

In [ ]:
df_double_label = df_double_feat['label_death'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
df_double_feat = df_double_feat[['cellMass', 'dnaMass', 'membrane_mass', 'extracellular_mass']]

In [ ]:
cnf_double = confusion_matrix(df_double_label, clf.predict(df_double_feat))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_double,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
print(cnf_double)
disp.plot(cmap=colors, xticks_rotation='vertical', )
disp.figure_.savefig('confusion_matrix_doubleKO_init.png', bbox_inches="tight", dpi=600)


In [ ]:
print('F1-score on the double KOs is ' + str(f1_score(df_double_label, clf.predict(df_double_feat),  pos_label='essential')))

# Test on multiple KOs

## Apply model

In [ ]:
from itertools import chain
df_multiple_test = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_multiple_test_perms_final.pkl')
df_multiple_train = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_multiple_train_perms_final.pkl')


In [ ]:
df_multiple_train = df_multiple_train.drop_duplicates(subset='Combos')
df_multiple_test = df_multiple_test.drop_duplicates(subset='Combos')


In [ ]:
set(list(chain(*[x.split() for x in list(df_multiple_train['Combos'])])))

In [ ]:
set(list(chain(*[x.split() for x in list(df_multiple_test['Combos'])])))

In [ ]:
df_multiple = pd.concat([df_multiple_train, df_multiple_test])
df_multiple = df_multiple.rename(columns={"label": "label_death"})
df_multiple_label = df_multiple['label_death'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
df_multiple_feat = df_multiple[['cellMass', 'dnaMass', 'membrane_mass', 'extracellular_mass']]

In [ ]:
plt.rcParams["figure.figsize"] = (1.8, 2)
font = {'weight' : 'normal', 'size':14} 
plt.rc('font', **font)   
cnf_multiple = confusion_matrix(df_multiple_label, clf.predict(df_multiple_feat))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_multiple,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
print(cnf_multiple)
disp.plot(cmap=colors, xticks_rotation='vertical', )
disp.figure_.savefig('confusion_matrix_multipleKO_init.png', bbox_inches="tight", dpi=600)


In [ ]:
print('F1-score on the multiple KOs is ' + str(f1_score(df_multiple_label, clf.predict(df_multiple_feat),  pos_label='essential')))

## Plot the data for different data sets

In [ ]:
df_multiple = pd.concat([df_multiple_test, df_multiple_train])
df_multiple_label = df_multiple['label'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
df_minesweeper = df_multiple.copy()[['cellMass', 'dnaMass', 'membrane_mass', 'extracellular_mass', 'label']]

In [ ]:
font = {'weight' : 'normal', 'size':12} 
plt.rc('font', **font)                                                                                          
plt.rcParams["figure.figsize"] = (6.61,2.3)


df_deadly_single = df_single[df_single["label_death"]=='deadly_gene'][['dnaMass', 'cellMass']]
df_not_deadly_single = df_single[df_single["label_death"]=='not_deadly'][['dnaMass', 'cellMass']]

df_deadly_double = df_double[df_double["label_death"]=='deadly_gene'][['dnaMass', 'cellMass']]
df_not_deadly_double = df_double[df_double["label_death"]=='not_deadly'][['dnaMass', 'cellMass']]

df_deadly_multi = df_minesweeper[df_minesweeper["label"]=='deadly_gene'][['dnaMass', 'cellMass']].dropna().astype(float)
df_not_deadly_multi = df_minesweeper[df_minesweeper["label"]=='not_deadly'][['dnaMass', 'cellMass']].dropna().astype(float)


fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
#plt.figure(figsize=(4.91, 2.41))


ax1.scatter(df_deadly_single['cellMass'].str[-1], df_deadly_single['dnaMass'].str[-1], c='darkorange', s=50, alpha=0.2, marker='o')
ax1.scatter(df_not_deadly_single['cellMass'].str[-1], df_not_deadly_single['dnaMass'].str[-1], c='b', s=50, alpha = 0.2, marker='x')
ax1.set_xlim(1500,3000)
ax1.set_ylim(8,14)
ax1.set_xlabel('Cell mass (fg)')
ax1.set_title('Single-gene KOs', fontsize=12)
ax1.tick_params(axis='both', which='major', labelsize=12)


ax2.scatter(df_deadly_double['cellMass'].str[-1], df_deadly_double['dnaMass'].str[-1], c='darkorange', s=50, alpha=0.2, marker='o')
ax2.scatter(df_not_deadly_double['cellMass'].str[-1], df_not_deadly_double['dnaMass'].str[-1], c='b', s=50, alpha = 0.2, marker='x')
ax2.scatter(df_not_deadly_double['cellMass'].iloc[0][-1], df_not_deadly_double['dnaMass'].iloc[0][-1], c='b', s=50, alpha = 0.8, marker='x', label='Non-deadly KO')
ax2.scatter(df_deadly_double['cellMass'].iloc[0][-1], df_deadly_double['dnaMass'].iloc[0][-1], c='darkorange', s=50, alpha=0.8, marker='o', label='Deadly KO')
ax2.set_xlim(1500,3000)
ax2.set_ylim(8,14)
ax2.set_xlabel('Cell mass (fg)')
ax2.set_title('Double-gene KOs', fontsize=12)
ax2.tick_params(axis='both', which='major', labelsize=12)

print(df_deadly_multi['cellMass'])
ax3.scatter(df_deadly_multi['cellMass'], df_deadly_multi['dnaMass'], c='darkorange', s=50, alpha=0.2, marker='o')
ax3.scatter(df_not_deadly_multi['cellMass'], df_not_deadly_multi['dnaMass'], c='b', s=50, alpha = 0.2, marker='x')
ax3.scatter(df_deadly_multi['cellMass'].iloc[0], df_deadly_multi['dnaMass'].iloc[0], c='b', s=50, alpha = 0.8, marker='x', label='Non-deadly KO')
ax3.scatter(df_not_deadly_multi['cellMass'].iloc[0], df_not_deadly_multi['dnaMass'].iloc[0], c='darkorange', s=50, alpha=0.8, marker='o', label='Deadly KO')
ax3.set_xlim(1500,3000)
ax3.set_ylim(8,14)
ax3.set_xlabel('Cell mass (fg)')
ax3.set_title('Multiple-gene KOs', fontsize=12)
ax3.tick_params(axis='both', which='major', labelsize=12)
ax3.legend(loc='lower right', fontsize=8.5)

fig.tight_layout()
#fig.text(0.5, 0.015, 'Cell mass (fg)', ha='center', fontsize ='medium')
fig.text(0, 0.5, 'DNA mass (fg)', va='center', rotation='vertical', fontsize='medium')
fig.savefig('top_feat_distrib.png', dpi=800)

# Train with multiple KOs added to training set

In [ ]:
df_multiple_train = df_multiple_train.drop_duplicates(subset=["cell_path"])


In [ ]:
df_multiple_train[df_multiple_train['KO_id'].isna()]['cell_path']

In [ ]:
combos_genes = pd.read_csv('/home/ig13470/Documents/5_percent_segments.csv', sep='\t', header=None)
combos_genes = combos_genes.rename(columns={0: "Segment", 1: "Genes"})

In [ ]:
for ind, row in df_multiple_train.iterrows():
    genes = []
    for segment in row['Combos'].split(' '):
        genes.extend(combos_genes[combos_genes['Segment']==segment]['Genes'].values[0].split(' '))
    df_multiple_train.loc[ind, 'Genes'] = str(genes[:-1])

In [ ]:
df_multiple_train.to_csv('multiple_ko_train.csv')

In [ ]:
df_multiple_train = df_multiple_train.rename(columns={"label": "label_death"})
df_multiple_train = df_multiple_train[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
                                       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
                                       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
                                        'label_death', 'cell_path']]

In [ ]:
df_multiple_train = df_multiple_train[~df_multiple_train.isna().any(axis=1)]
df_multiple_train['label_death'] = df_multiple_train['label_death'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})


In [ ]:
font = {'weight' : 'normal', 'size':20} 
plt.rc('font', **font)    
plt.rcParams["figure.figsize"] = (2.45, 2.45)


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = df_features.drop([ 'label', 'cell', 'seed', 'instantaniousGrowthRate',
                       'label_lifetime', 'generation', 'gene_name'], axis=1)
y=df_features.label_death.replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
gs = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=6)

gen = list(next(gs.split(X, y, groups=X.gene_id)))
train_ix = gen[0]
test_ix = gen[1]

X_train = X.iloc[train_ix].drop(columns=['label_death'])
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix].drop(columns=['label_death'])
y_test = y.iloc[test_ix]

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
df_multiple_X = df_multiple_train[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
       'smallMoleculeMass', 'membrane_mass']]
df_multiple_y = df_multiple_train['label_death']

In [ ]:
from sklearn.utils import shuffle

X_train = pd.concat([df_multiple_X, X_train])
y_train = pd.concat([df_multiple_y, y_train])



In [ ]:
X_train.columns

In [ ]:

df_multiple_label_test = df_multiple_test['label'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
df_multiple_test = df_multiple_test[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
       'smallMoleculeMass', 'membrane_mass']]

In [ ]:
y_train.value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
import itertools

tr_acc=[]
test_single_acc= []
test_multiple_acc = []
hyperparams = []

#for x in itertools.product(list(range(2,20)), list(range(2,20))):
#    hyperparams.append(x)
clf = RandomForestClassifier(max_depth=6, n_estimators=3, random_state=0, class_weight='balanced')
clf.fit(X_train.drop(columns=['cell_path', 'gene_id']), y_train)

print('Confussion matrix for the training set')
cnf_tr = confusion_matrix(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_tr,
                               display_labels=['Deadly KO', 'Non-deadly KO'])

disp.plot(cmap=colors, xticks_rotation='vertical')
disp.figure_.savefig('confusion_matrix_training_retrained.png', bbox_inches="tight", dpi=600)
plt.show()
print('F1-score training ', f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
tr_acc.append(f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
print('confusion matrix for the test set')
cnf_test = confusion_matrix(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_test,
                               display_labels=['Deadly KO', 'Non-deadly KO'])

disp.plot(cmap=colors, xticks_rotation='vertical')
disp.figure_.savefig('confusion_matrix_test_retrained.png', bbox_inches="tight", dpi=600)
plt.show()
print('F1 score for the test set is ' + str(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential')))
test_single_acc.append(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))

print('F1 score for the multiple set is ' + str(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential')))
test_multiple_acc.append(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential'))

In [ ]:
plt.rcParams["figure.figsize"] = (8, 2.32)

features = X_train.columns
f_i = list(zip(features,clf.feature_importances_))
f_i.sort(key = lambda x : x[1], reverse=True)
to_d = [t[0] for t in f_i if t[1]==0]
top_30 = [t[0] for t in f_i[:10]]
plt.bar(*zip(*f_i))
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Feature importance', fontsize=12)
plt.savefig('feat_importance_rf_retrained.png', bbox_inches="tight")
plt.savefig('feat_importance_rf_retrained.eps', bbox_inches="tight", format='eps')
plt.show()

# Test on double KOs

In [ ]:
cols = ['cellMass', 'growth', 'dryMass', 'waterMass', 'dnaMass', 'cytosol_mass',
       'tRnaMass', 'extracellular_mass', 'rRnaMass', 'proteinMass',
       'projection_mass', 'rnaMass', 'outer_membrane_mass', 'flagellum',
       'pilus_mass', 'cellVolume', 'inner_membrane_mass', 'mRnaMass',
       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
       'periplasm_mass']

#this is for the cells that survive only for 1 time-step
df_double1 = df_double[df_double['cellMass'].str.len().isna()]
for c in cols:
    df_double1[c] = df_double1[c].astype(float)
    

#this is for the cells that survive for more than 1 time-step
df_double2 = df_double[~df_double['cellMass'].str.len().isna()]

for c in cols:
    df_double2[c] = df_double2[c].apply(lambda x:x[-1])


        
df_double_feat = pd.concat([df_double1, df_double2])
df_double_feat['growth'] = df_double_feat['growth'].fillna(0)
df_double_feat = df_double_feat.drop(columns=['processMassDifferences', 'time'])[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
       'smallMoleculeMass', 'membrane_mass']]



In [ ]:

cnf_double = confusion_matrix(df_double_label, clf.predict(df_double_feat))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_double,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
print(cnf_double)
disp.plot(cmap=colors, xticks_rotation='vertical', )
disp.figure_.savefig('confusion_matrix_doubleKO_retrain.png', bbox_inches="tight", dpi=600)

print('F1 score for the double set is ' + str(f1_score(df_double_label, clf.predict(df_double_feat),  pos_label='essential')))

# Test on multiple KOs

In [ ]:
df_multiple_test = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_multiple_test_perms_final.pkl')
df_multiple_test = df_multiple_test.drop_duplicates(subset='Combos')[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
       'smallMoleculeMass', 'membrane_mass']]


In [ ]:
cnf_multiple = confusion_matrix(df_multiple_label_test, clf.predict(df_multiple_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_multiple,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
print(cnf_multiple)
disp.plot(cmap=colors, xticks_rotation='vertical', )
disp.figure_.savefig('confusion_matrix_multipleKO_test_final.png', bbox_inches="tight", dpi=600)


print('F1 score for the multiple set is ' + str(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential')))

In [ ]:
# save the model to disk
filename = 'rf_model_surrogate_retrained_new.sav'
pickle.dump(clf, open(filename, 'wb')) 

# Add more data to the training set as interations go along

In [ ]:
df_multiple_train = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_multiple_train_perms_final.pkl')

df_multiple_train_st4_blue3 = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_for_retraining_stage4_blue3.pkl')
df_multiple_train_st4_red1 = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_for_retraining_stage4_red1.pkl')
df_multiple_train_st4_red3 = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_for_retraining_stage4_red3.pkl')
df_multiple_train_st4_yellow3 = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_for_retraining_stage4_yellow3.pkl')

df_multiple_train_st4 = pd.concat([df_multiple_train_st4_blue3, df_multiple_train_st4_red1, df_multiple_train_st4_red3, df_multiple_train_st4_yellow3])

In [ ]:
len(df_multiple_train_st4)

In [ ]:
df_multiple_train = pd.concat([df_multiple_train, df_multiple_train_st4])

In [ ]:
df_multiple_train = df_multiple_train.rename(columns={"label": "label_death"})
df_multiple_train = df_multiple_train[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
                                       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
                                       'smallMoleculeMass', 'instantaniousGrowthRate', 'membrane_mass',
                                        'label_death', 'cell_path']]

In [ ]:
df_multiple_train = df_multiple_train[~df_multiple_train.isna().any(axis=1)]
df_multiple_train['label_death'] = df_multiple_train['label_death'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})


In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = df_features.drop([ 'label', 'cell', 'seed', 'instantaniousGrowthRate',
                       'label_lifetime', 'generation', 'gene_name'], axis=1)
y=df_features.label_death.replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
gs = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=6)

gen = list(next(gs.split(X, y, groups=X.gene_id)))
train_ix = gen[0]
test_ix = gen[1]

X_train = X.iloc[train_ix].drop(columns=['label_death'])
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix].drop(columns=['label_death'])
y_test = y.iloc[test_ix]

In [ ]:
df_multiple_X = df_multiple_train[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
       'smallMoleculeMass', 'membrane_mass']]
df_multiple_y = df_multiple_train['label_death']

In [ ]:
from sklearn.utils import shuffle

X_train = pd.concat([df_multiple_X, X_train])
y_train = pd.concat([df_multiple_y, y_train])

In [ ]:
y_train.value_counts()

In [ ]:
df_multiple_test = pd.read_pickle('/home/ig13470/Documents/WCM_data/df_multiple_test_perms_final.pkl')

df_multiple_label_test = df_multiple_test['label'].replace({'not_deadly': 'non_essential', 'deadly_gene': 'essential'})
df_multiple_test = df_multiple_test[['cellMass', 'growth', 'dnaMass', 'tRnaMass', 'extracellular_mass',
       'proteinMass', 'projection_mass', 'pilus_mass', 'mRnaMass',
       'smallMoleculeMass', 'membrane_mass']]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
import itertools

tr_acc=[]
test_single_acc= []
test_multiple_acc = []
hyperparams = []

for x in itertools.product(list(range(2,20)), list(range(2,20))):
    hyperparams.append(x)
    clf = RandomForestClassifier(max_depth=x[0], n_estimators=x[1], random_state=14)
    clf.fit(X_train.drop(columns=['cell_path', 'gene_id']), y_train)
    
    print('Confussion matrix for the training set')
    cnf_tr = confusion_matrix(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])))
    disp = ConfusionMatrixDisplay(confusion_matrix=cnf_tr,
                                   display_labels=['Deadly KO', 'Non-deadly KO'])
    
    #disp.plot(cmap=colors, xticks_rotation='vertical')
    #disp.figure_.savefig('confusion_matrix_training_retrained.png', bbox_inches="tight", dpi=600)
    #plt.show()
    print('F1-score training ', f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
    tr_acc.append(f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
    print('confusion matrix for the test set')
    cnf_test = confusion_matrix(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])))
    disp = ConfusionMatrixDisplay(confusion_matrix=cnf_test,
                                   display_labels=['Deadly KO', 'Non-deadly KO'])
    
    #disp.plot(cmap=colors, xticks_rotation='vertical')
    #disp.figure_.savefig('confusion_matrix_test_retrained.png', bbox_inches="tight", dpi=600)
    #plt.show()
    print('F1 score for the test set is ' + str(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential')))
    test_single_acc.append(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
    
    print('F1 score for the multiple set is ' + str(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential')))
    test_multiple_acc.append(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential'))

In [ ]:
index_max = np.argmax(test_multiple_acc)

In [ ]:
print(test_multiple_acc[index_max], test_single_acc[index_max], tr_acc[index_max])
print(hyperparams[index_max])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
import itertools

tr_acc=[]
test_single_acc= []
test_multiple_acc = []
hyperparams = []

#for x in itertools.product(list(range(2,20)), list(range(2,20))):
clf = RandomForestClassifier(max_depth=3, n_estimators=16, random_state=14)
clf.fit(X_train.drop(columns=['cell_path', 'gene_id']), y_train)

print('Confussion matrix for the training set')
cnf_tr = confusion_matrix(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_tr,
                               display_labels=['Deadly KO', 'Non-deadly KO'])

disp.plot(cmap=colors, xticks_rotation='vertical')
disp.figure_.savefig('confusion_matrix_training_retrained.png', bbox_inches="tight", dpi=600)
plt.show()
print('F1-score training ', f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
tr_acc.append(f1_score(y_train, clf.predict(X_train.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))
print('confusion matrix for the test set')
cnf_test = confusion_matrix(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_test,
                               display_labels=['Deadly KO', 'Non-deadly KO'])

disp.plot(cmap=colors, xticks_rotation='vertical')
disp.figure_.savefig('confusion_matrix_test_retrained.png', bbox_inches="tight", dpi=600)
plt.show()
print('F1 score for the test set is ' + str(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential')))
test_single_acc.append(f1_score(y_test, clf.predict(X_test.drop(columns=['cell_path', 'gene_id'])),  pos_label='essential'))

print('F1 score for the multiple set is ' + str(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential')))
test_multiple_acc.append(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential'))

In [ ]:
cnf_multiple = confusion_matrix(df_multiple_label_test, clf.predict(df_multiple_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_multiple,
                               display_labels=['Deadly KO', 'Non-deadly KO'])
print(cnf_multiple)
disp.plot(cmap=colors, xticks_rotation='vertical', )
disp.figure_.savefig('confusion_matrix_multipleKO_test_final.png', bbox_inches="tight", dpi=600)


print('F1 score for the multiple set is ' + str(f1_score(df_multiple_label_test, clf.predict(df_multiple_test),  pos_label='essential')))

In [ ]:
# save the model to disk
filename = 'rf_model_surrogate_retrained_stage4.sav'
pickle.dump(clf, open(filename, 'wb')) 